<a href="https://colab.research.google.com/github/guifzy/analise_estatistica_dados/blob/main/teste_tukey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_excel('/content/dic.xls')
print("Dimensões do dataset:", data.shape)

Dimensões do dataset: (20, 3)


In [ ]:
data.head(20)

,Variedade,Repeticao,Peso
0,A,1,78
1,A,2,88
2,A,3,72
3,A,4,74
4,A,5,98
5,B,1,79
6,B,2,56
7,B,3,71
8,B,4,96
9,B,5,55


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Variedade  20 non-null     object
 1   Repeticao  20 non-null     int64 
 2   Peso       20 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 608.0+ bytes


In [ ]:
data.isnull().sum()

,0
Variedade,0
Repeticao,0
Peso,0


In [ ]:
data['Variedade'].value_counts()

,count
Variedade,
A,5
B,5
C,5
D,5


# transformando em fatores

In [ ]:
data_encoded = pd.get_dummies(data)

data_encoded.head()

,Repeticao,Peso,Variedade_A,Variedade_B,Variedade_C,Variedade_D
0,1,78,True,False,False,False
1,2,88,True,False,False,False
2,3,72,True,False,False,False
3,4,74,True,False,False,False
4,5,98,True,False,False,False


In [ ]:
data_encoded.info()
lista = ['Variedade_A', 'Variedade_B', 'Variedade_C', 'Variedade_D']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Repeticao    20 non-null     int64
 1   Peso         20 non-null     int64
 2   Variedade_A  20 non-null     bool 
 3   Variedade_B  20 non-null     bool 
 4   Variedade_C  20 non-null     bool 
 5   Variedade_D  20 non-null     bool 
dtypes: bool(4), int64(2)
memory usage: 528.0 bytes


In [ ]:
data_encoded = data_encoded[lista].astype(int)

In [ ]:
data_encoded.head(20)

,Variedade_A,Variedade_B,Variedade_C,Variedade_D
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
5,0,1,0,0
6,0,1,0,0
7,0,1,0,0
8,0,1,0,0
9,0,1,0,0


In [ ]:
data_final = data.join(data_encoded)
data_f = data_final.drop("Variedade", axis=1)
data_f.head()

,Repeticao,Peso,Variedade_A,Variedade_B,Variedade_C,Variedade_D
0,1,78,1,0,0,0
1,2,88,1,0,0,0
2,3,72,1,0,0,0
3,4,74,1,0,0,0
4,5,98,1,0,0,0


In [ ]:
# prompt: me ajude a multiplicar o valor da coluna peso em relação a quantas vezes ele se repete em sua respectiva linha da coluna Repeticao

data_f['Peso_Repetido'] = data_f['Peso'] * data_f['Repeticao']
data_f.head()

,Repeticao,Peso,Variedade_A,Variedade_B,Variedade_C,Variedade_D,Peso_Repetido
0,1,78,1,0,0,0,78
1,2,88,1,0,0,0,176
2,3,72,1,0,0,0,216
3,4,74,1,0,0,0,296
4,5,98,1,0,0,0,490


In [ ]:
# Realize o teste ANOVA entre os grupos
from scipy import stats
fvalue, pvalue = stats.f_oneway(data_f[data_f['Variedade_A'] == 1]['Peso'],
                                data_f[data_f['Variedade_B'] == 1]['Peso'],
                                data_f[data_f['Variedade_C'] == 1]['Peso'],
                                data_f[data_f['Variedade_D'] == 1]['Peso'])
print("\nResultado do teste ANOVA:")
print("F-value:", fvalue)
print("P-value:", pvalue)


Resultado do teste ANOVA:
F-value: 3.7754189198210826
P-value: 0.031881023385176496


$H_0$ = as medias dos grupos são iguais

$H_1$ = Pelo menos uma media é diferente

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# post-hoc test
tukey_result = pairwise_tukeyhsd(data_f['Peso'], data_f[['Variedade_A', 'Variedade_B', 'Variedade_C', 'Variedade_D']].idxmax(axis=1), alpha=0.05)

print("\nResultado do teste de Tukey:")
print(tukey_result)


Resultado do teste de Tukey:
      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1      group2   meandiff p-adj   lower    upper  reject
---------------------------------------------------------------
Variedade_A Variedade_B    -10.6 0.4602 -30.8163  9.6163  False
Variedade_A Variedade_C    -16.6 0.1282 -36.8163  3.6163  False
Variedade_A Variedade_D    -22.8 0.0245 -43.0163 -2.5837   True
Variedade_B Variedade_C     -6.0 0.8303 -26.2163 14.2163  False
Variedade_B Variedade_D    -12.2 0.3429 -32.4163  8.0163  False
Variedade_C Variedade_D     -6.2 0.8164 -26.4163 14.0163  False
---------------------------------------------------------------


# Novo teste

In [ ]:
data = {
    'Variedade': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C'],
    'Peso': [5, 7, 6, 8, 9, 6, 5, 8, 7, 10, 9, 11, 10, 12, 10]
}
data = pd.DataFrame(data)
print("Dimensões do dataset:", data.shape)

Dimensões do dataset: (15, 2)


In [ ]:
data.head(20)

,Variedade,Peso
0,A,5
1,A,7
2,A,6
3,A,8
4,A,9
5,B,6
6,B,5
7,B,8
8,B,7
9,B,10


In [ ]:
import pandas as pd
from scipy import stats

# Dados
data = {
    'Variedade': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C'],
    'Peso': [5, 7, 6, 8, 9, 6, 5, 8, 7, 10, 9, 11, 10, 12, 10]
}

# Criação do DataFrame
df = pd.DataFrame(data)

# Codificação das variáveis
df_encoded = pd.get_dummies(df, columns=['Variedade'], drop_first=False)

# Realizar o teste ANOVA
fvalue, pvalue = stats.f_oneway(

    df_encoded[df_encoded['Variedade_A'] == 1]['Peso'],
    df_encoded[df_encoded['Variedade_B'] == 1]['Peso'],
    df_encoded[df_encoded['Variedade_C'] == 1]['Peso']
)

print("F-value:", fvalue)
print("P-value:", pvalue)


F-value: 7.280000000000005
P-value: 0.008505876095950838


In [ ]:
df_encoded.head(20)

,Peso,Variedade_B,Variedade_C
0,5,False,False
1,7,False,False
2,6,False,False
3,8,False,False
4,9,False,False
5,6,True,False
6,5,True,False
7,8,True,False
8,7,True,False
9,10,True,False


In [ ]:
data_final.head(20)

,Variedade,Peso,Variedade_A,Variedade_B,Variedade_C
0,A,5,1,0,0
1,A,7,1,0,0
2,A,6,1,0,0
3,A,8,1,0,0
4,A,9,1,0,0
5,B,6,0,1,0
6,B,5,0,1,0
7,B,8,0,1,0
8,B,7,0,1,0
9,B,10,0,1,0


In [ ]:
from scipy import stats

fvalue, pvalue = stats.f_oneway(
    data_f[data_f['Variedade_A'] == 1]['Peso'],
    data_f[data_f['Variedade_B'] == 1]['Peso'],
    data_f[data_f['Variedade_C'] == 1]['Peso'])

print("\nResultado do teste ANOVA:")
print("F-value:", fvalue)
print("P-value:", pvalue)


Resultado do teste ANOVA:
F-value: 7.280000000000005
P-value: 0.008505876095950838


In [ ]:
# df_between = número de grupos - 1
df_b = len(data_final['Variedade'].unique()) - 1

# df_within = número total de observações - número de grupos
df_w = len(data_final) - len(data_final['Variedade'].unique())

print("\nGraus de liberdade:")
print("Entre grupos (df_between):", df_b)
print("Dentro dos grupos (df_within):", df_w)



Graus de liberdade:
Entre grupos (df_between): 2
Dentro dos grupos (df_within): 12


In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
alpha = 0.05
if pvalue > alpha:
  print("Não há evidência suficiente para rejeitar a hipótese nula.")
else:
  print("Há evidência suficiente para rejeitar a hipótese nula.")
  # post-hoc test
  tukey_result = pairwise_tukeyhsd(data_f['Peso'], data_f[['Variedade_A', 'Variedade_B', 'Variedade_C']].idxmax(axis=1), alpha=alpha)

  print("\nResultado do teste de Tukey:")
  print(tukey_result)

Há evidência suficiente para rejeitar a hipótese nula.

Resultado do teste de Tukey:
     Multiple Comparison of Means - Tukey HSD, FWER=0.05     
   group1      group2   meandiff p-adj   lower  upper  reject
-------------------------------------------------------------
Variedade_A Variedade_B      0.2 0.9782 -2.4679 2.8679  False
Variedade_A Variedade_C      3.4 0.0135  0.7321 6.0679   True
Variedade_B Variedade_C      3.2 0.0194  0.5321 5.8679   True
-------------------------------------------------------------
